In [1]:
import torch
import pandas as pd
import numpy as np

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 2060


In [2]:
from torch.utils.data.sampler import SequentialSampler
from torch.utils.data.dataloader import DataLoader
# from torch.utils.data import Dataset

class ClosePriceDataset(torch.utils.data.Dataset):
    """Some Information about ClosePriceDataset"""
    def __init__(self):
        super(ClosePriceDataset, self).__init__()
        data = pd.read_csv('../datasets/BRK-A.csv')
        n_steps = 5
        self.x, self.y = self.split_sequence(data['Close'], n_steps)
        self.n_sample = self.x.size(dim=0)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_sample

    def split_sequence(slef, sequence: pd.Series, timestep: int):
        X = list()
        y = list()

        for i in range(len(sequence)):
            end_idx = i + timestep
            if end_idx > len(sequence) - 1:
                break
            seq_X, seq_y = [sequence[i:end_idx]], sequence[end_idx]
            X.append(seq_X)
            y.append(seq_y)


        return torch.FloatTensor(X), torch.FloatTensor(y)


def data_split(dataset, val_split = 0.25, random_seed = 0):
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    val_idx = int(np.floor(dataset_size * val_split))
    train_indices, val_indices = indices[val_idx:], indices[:val_idx]
    train_sampler = SequentialSampler(train_indices)
    valid_sampler = SequentialSampler(val_indices)

    return train_sampler, valid_sampler


batch_size = 128
val_split = 0.25
random_seed= 42

dataset = ClosePriceDataset()
train_sampler, valid_sampler = data_split(dataset=dataset, val_split=val_split, random_seed=random_seed)
train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

dataiter = iter(train_loader)
data = dataiter.next()
features, labels = data
print(features, labels)
print(features.shape, labels.shape)


tensor([[[290., 290., 290., 290., 290.]],

        [[290., 290., 290., 290., 270.]],

        [[290., 290., 290., 270., 270.]],

        [[290., 290., 270., 270., 270.]],

        [[290., 270., 270., 270., 270.]],

        [[270., 270., 270., 270., 270.]],

        [[270., 270., 270., 270., 260.]],

        [[270., 270., 270., 260., 260.]],

        [[270., 270., 260., 260., 260.]],

        [[270., 260., 260., 260., 260.]],

        [[260., 260., 260., 260., 265.]],

        [[260., 260., 260., 265., 265.]],

        [[260., 260., 265., 265., 265.]],

        [[260., 265., 265., 265., 260.]],

        [[265., 265., 265., 260., 260.]],

        [[265., 265., 260., 260., 255.]],

        [[265., 260., 260., 255., 245.]],

        [[260., 260., 255., 245., 245.]],

        [[260., 255., 245., 245., 255.]],

        [[255., 245., 245., 255., 255.]],

        [[245., 245., 255., 255., 260.]],

        [[245., 255., 255., 260., 260.]],

        [[255., 255., 260., 260., 275.]],

        [[2

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ConvNet(nn.Module):
    """Some Information about MyModule"""
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, groups=1, bias=True, kernel_size=2, stride=1)
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(2 * 64, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = self.maxpool1(F.relu(self.conv1(x)))
        # print('MaxPool1 Shape: {}'.format(x.shape))
        x = torch.flatten(x, start_dim=1)
        # print('Flaten Shape: {}'.format(x.shape))
        x = F.relu(self.fc1(x))
        # print('Linear1 Shape: {}'.format(x.shape))
        x = self.fc2(x)
        
        return x

net = ConvNet()
print(net)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())

ConvNet(
  (conv1): Conv1d(1, 64, kernel_size=(2,), stride=(1,))
  (maxpool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=1, bias=True)
)


In [4]:
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        # inputs = torch.from_numpy(x)
        # labels = torch.from_numpy(y)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 64 == 0:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 5:.3f}')
            running_loss = 0.0

print('Finished Training')

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1,     1] loss: 23056.034


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([68])) that is different to the input size (torch.Size([68, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[2,     1] loss: 749.317
[3,     1] loss: 747.980
[4,     1] loss: 747.552
[5,     1] loss: 748.636
[6,     1] loss: 749.139
[7,     1] loss: 749.493
[8,     1] loss: 749.746
[9,     1] loss: 749.931
[10,     1] loss: 750.066
[11,     1] loss: 750.169
[12,     1] loss: 750.248
[13,     1] loss: 750.309
[14,     1] loss: 750.358
[15,     1] loss: 750.396
[16,     1] loss: 750.428
[17,     1] loss: 750.453
[18,     1] loss: 750.473
[19,     1] loss: 750.493
[20,     1] loss: 750.505
[21,     1] loss: 750.516
[22,     1] loss: 750.525
[23,     1] loss: 750.534
[24,     1] loss: 750.541
[25,     1] loss: 750.547
[26,     1] loss: 750.553
[27,     1] loss: 750.559


KeyboardInterrupt: 